**Importamos las librerias que necestiaremos**
- usamos:
- python 3.13.1
- numpy 2.3.3
- pip 25.2
- matplotlib 3.10.6
- statsmodel 0.14.5
- STL 0.0.3

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from statsmodels.stats.outliers_influence import variance_inflation_factor

**Cargamos nuestra base de datos en un frame de Pandas**

In [ ]:
# Subimos nuestra base de datos, cargamos ambos datasets
df_1 = pd.read_csv("synthetic_mmm_weekly_india_SAT.csv")
df_2 = pd.read_csv("synthetic_mmm_weekly_india.csv")

# Visualizamos el frame
df_1


**Revisamos las carracterisitcas generales del dataset**

In [94]:
# Dimenciones  
print(f"Sus dimensiones son:\n\ndf_1: {df_1.shape}\ndf_2: {df_2.shape}")
print(" ")

# comparación de nombres de columnas (ignorando el orden)
print("Tienen la misma cantidad de columnas con los mismos nombres:", set(df_1.columns) == set(df_2.columns))
print(" ")


Sus dimensiones son:

df_1: (11232, 28)
df_2: (11232, 28)
 
Tienen la misma cantidad de columnas con los mismos nombres: True
 


**Buscamos:**
- Tipos de datos. 
- Valores nulos.
- Valores duplicados. 
- Uso de memoria.

In [95]:
print(f"Informacion del primer data set:---------------------\n")
df_1.info()
print(" ")

print(f"Informacion del segundo data set:---------------------\n")
df_2.info()
print(" ")

# Verificamos si hay valores nulos en ambos DataFrames
print(f"Valores nulos en df_1:\n{df_1.isnull().sum()}\n")
print(f"Valores nulos en df_2:\n{df_2.isnull().sum()}\n")

# Veriuficamos si hay duplicados en ambos DataFrames    
print(f"Duplicados en df_1: {df_1.duplicated().sum()}")
print(f"Duplicados en df_2: {df_2.duplicated().sum()}")



Informacion del primer data set:---------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11232 entries, 0 to 11231
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Week                     11232 non-null  object 
 1   Geo                      11232 non-null  object 
 2   Brand                    11232 non-null  object 
 3   SKU                      11232 non-null  object 
 4   Sales_Units              11232 non-null  float64
 5   Sales_Value              11232 non-null  float64
 6   MRP                      11232 non-null  float64
 7   Net_Price                11232 non-null  float64
 8   Feature_Flag             11232 non-null  int64  
 9   Display_Flag             11232 non-null  int64  
 10  TPR_Flag                 11232 non-null  int64  
 11  Trade_Spend              11232 non-null  float64
 12  TV_Impressions           11232 non-null  float64
 13  YouTube_Impressions  

**Cambiamos el tipo de dato de la columna "week"**

In [ ]:
df_1["Week"] = pd.to_datetime(df_1["Week"], errors="coerce")
df_2["Week"] = pd.to_datetime(df_2["Week"], errors="coerce")

# Verificamos si hay valores nulos en ambos DataFrames
print("NaT en df_1.Week:", df_1["Week"].isna().sum())
print("NaT en df_2.Week:", df_2["Week"].isna().sum())
print(" ")

# Revisamos el cambio
print(f"Informacion del primer data set:---------------------\n")
df_1.info()
print(" ")

print(f"Informacion del segundo data set:---------------------\n")
df_2.info()
print(" ")

NaT en df_1.Week: 0
NaT en df_2.Week: 0
 
Informacion del primer data set:---------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11232 entries, 0 to 11231
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Week                     11232 non-null  datetime64[ns]
 1   Geo                      11232 non-null  object        
 2   Brand                    11232 non-null  object        
 3   SKU                      11232 non-null  object        
 4   Sales_Units              11232 non-null  float64       
 5   Sales_Value              11232 non-null  float64       
 6   MRP                      11232 non-null  float64       
 7   Net_Price                11232 non-null  float64       
 8   Feature_Flag             11232 non-null  int64         
 9   Display_Flag             11232 non-null  int64         
 10  TPR_Flag                 11232 non-null  int64         
 

- Notamos que tenemos datos **"object"** asi que para reducir el espacio en memoria los cambiaremos a **"category"**
- Estandarizaremos el texto a todos en mayusculas

In [100]:
# Selecciona explícitamente 3 columnas por nombre
cols = ['Geo','Brand','SKU']
df_1[cols]
print(df_1[cols])

# Estandarizamos los textos a mayusculas y cambiamos a tipo "category"
for col in cols:
    df_1[col] = df_1[col].str.upper().astype('category')
    df_2[col] = df_2[col].str.upper().astype('category')

# Revisamos el cambio
print(f"Informacion del primer data set:---------------------\n")
df_1.info()
print(" ")
print(f"Informacion del segundo data set:---------------------\n")
df_2.info()





           Geo   Brand          SKU
0      CENTRAL  BRANDA  BRANDA_SKU1
1      CENTRAL  BRANDA  BRANDA_SKU2
2      CENTRAL  BRANDA  BRANDA_SKU3
3      CENTRAL  BRANDB  BRANDB_SKU1
4      CENTRAL  BRANDB  BRANDB_SKU2
...        ...     ...          ...
11227     WEST  BRANDB  BRANDB_SKU2
11228     WEST  BRANDB  BRANDB_SKU3
11229     WEST  BRANDC  BRANDC_SKU1
11230     WEST  BRANDC  BRANDC_SKU2
11231     WEST  BRANDC  BRANDC_SKU3

[11232 rows x 3 columns]
Informacion del primer data set:---------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11232 entries, 0 to 11231
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Week                     11232 non-null  object  
 1   Geo                      11232 non-null  category
 2   Brand                    11232 non-null  category
 3   SKU                      11232 non-null  category
 4   Sales_Units              11232 non-null  fl

**Revisamos no tenerr valores extraños como --, "", NA, etc**

In [ ]:
#revisamos que no se hayan guardado valores " ", --, etc como nulos

# for i in df_1.columns:
#     if df_1[i].dtype == 'object':
#         print(f"Valores únicos en df_1 columna '{i}': {df_1[i].unique()}")
#         print(" ")


**Revisamos que ambas bases de datos no sean identicas**

In [102]:
# Reducimos la cantidad de decimales a 3 en ambos DataFrames
df_1

# Comparamos ambos DataFrames
comparison = df_1.equals(df_2)
print(f"¿Los DataFrames son idénticos? {comparison}")


¿Los DataFrames son idénticos? False


**Hacemos una descripcion generral de nuestra base de datos**

In [109]:
# Revisamos las características estadísticas de ambos DataFrames
print("Características estadísticas de df_1:")
df_1.describe(include='all')
# print(" ")
# print("Características estadísticas de df_2:")
# df_2.describe(include='all')

#hacemos un histograma usando para el eje x sales_value y para el eje y sales_units
# df_1['Sales_Value'].hist(bins=30)
# plt.title('Histograma de Sales_Value')
# plt.xlabel('Sales_Value')
# plt.ylabel('Frecuencia')
# plt.show()



Características estadísticas de df_1:


,Week,Geo,Brand,SKU,Sales_Units,Sales_Value,MRP,Net_Price,Feature_Flag,Display_Flag,...,FB_Banner_Content_Score,IG_Banner_Content_Score,Weighted_Distribution,Numeric_Distribution,TDP,NOS,CPI,GDP_Growth,Festival_Index,Rainfall_Index
count,11232,11232,11232,11232,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000,...,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000,11232.000000
unique,156,8,3,9,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2022-07-02,CENTRAL,BRANDA,BRANDA_SKU1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,72,1404,3744,1248,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,409.144276,32453.433540,79.687066,78.108492,0.186699,0.184295,...,70.372429,65.332445,0.646951,0.670508,65.049250,4.396893,132.280576,5.082094,1.683299,2.151666
std,NaN,NaN,NaN,NaN,306.305614,27000.516407,16.358434,16.438529,0.389687,0.387742,...,9.148711,12.803109,0.167785,0.171485,18.578824,0.787389,7.696734,0.554610,0.936699,1.583922
min,NaN,NaN,NaN,NaN,29.971872,2566.350247,52.906281,46.028997,0.000000,0.000000,...,48.560626,30.066035,0.312468,0.313900,27.705402,2.578997,114.873524,3.488797,0.934774,0.343123
25%,NaN,NaN,NaN,NaN,198.665547,14754.451778,63.358215,62.805069,0.000000,0.000000,...,63.573587,55.580581,0.512283,0.527513,50.389215,3.816038,126.449164,4.686955,0.976820,0.954376
50%,NaN,NaN,NaN,NaN,325.212428,24592.362351,79.790936,78.143183,0.000000,0.000000,...,70.205004,66.091909,0.636878,0.655696,63.424177,4.348179,132.438203,5.044538,1.139112,1.285723
75%,NaN,NaN,NaN,NaN,528.543722,40811.772659,93.308157,91.933471,0.000000,0.000000,...,76.468158,75.414566,0.766018,0.792686,77.774855,4.918074,137.780111,5.493578,2.477604,3.791379
